In [1]:
import torch
from torchvision.models import resnet50, ResNet50_Weights
import cv2
from torchvision.transforms import transforms
from PIL import Image
import os

weights = ResNet50_Weights.IMAGENET1K_V1
model = resnet50(weights=weights)
model.eval() 

preprocess = weights.transforms()

def image_loader(image_name):
    if not os.path.exists(image_name):
        print(f"ERROR: File not found at path: {image_name}")
        return None
        
    image = Image.open(image_name).convert("RGB")
    return image

In [ ]:
image_file = "glare_images/glare7mouse.jpg"
# image_file = "../image_results/pre_vulture.png"
# image_file = "glare_outputs/00003.png"
image = image_loader(image_file)

image_mod_file = "glare_outputs/00007.png"
# image_mod_file = "../image_results/post_vulture.png"
image_mod = image_loader(image_mod_file)

In [ ]:
input_tensor = preprocess(image)
input_tensor_2 = preprocess(image_mod)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_batch = input_tensor.unsqueeze(0).to(device)
model.to(device)

with torch.no_grad():
    output = model(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0)

predicted_index = torch.argmax(probabilities).item()

categories = weights.meta["categories"]
predicted_label = categories[predicted_index]

print("-" * 45)
print("Original Image:")
print(f"Predicted Label: **{predicted_label}**")
print(f"Probability: {probabilities[predicted_index].item():.4f}")
print("Top 5 Predicted Categories:")
top5_prob, top5_indices = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    predicted_index = top5_indices[i].item()
    predicted_label = categories[predicted_index]
    probability = top5_prob[i].item()
    print(f"  {i+1}. **{predicted_label}** (Probability: {probability:.4f})")
print("-" * 45)
#------------------------------------------------------------#
input_batch = input_tensor_2.unsqueeze(0).to(device)
model.to(device)

with torch.no_grad():
    output = model(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0)

predicted_index = torch.argmax(probabilities).item()

categories = weights.meta["categories"]
predicted_label = categories[predicted_index]
print("Processed Image:")
print(f"Predicted Label: **{predicted_label}**")
print(f"Probability: {probabilities[predicted_index].item():.4f}")

print("Top 5 Predicted Categories:")
top5_prob, top5_indices = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    predicted_index = top5_indices[i].item()
    predicted_label = categories[predicted_index]
    probability = top5_prob[i].item()
    print(f"  {i+1}. **{predicted_label}** (Probability: {probability:.4f})")
print("-" * 45)

---------------------------------------------
Preprocessed Image:
Predicted Label: **vulture**
Probability: 0.9634
Top 5 Predicted Categories:
  1. **vulture** (Probability: 0.9634)
  2. **kite** (Probability: 0.0126)
  3. **ruffed grouse** (Probability: 0.0086)
  4. **hen** (Probability: 0.0038)
  5. **partridge** (Probability: 0.0036)
---------------------------------------------
Processed Image:
Predicted Label: **vulture**
Probability: 0.9905
Top 5 Predicted Categories:
  1. **vulture** (Probability: 0.9905)
  2. **hen** (Probability: 0.0069)
  3. **kite** (Probability: 0.0008)
  4. **ruffed grouse** (Probability: 0.0005)
  5. **partridge** (Probability: 0.0003)
---------------------------------------------
